[View in Colaboratory](https://colab.research.google.com/github/kundajelab/ssvmimp/blob/master/lsgkm.ipynb)

In [2]:
#install lsgkm
! git clone https://github.com/Dongwon-Lee/lsgkm
% cd lsgkm/src
! make
%cd ../..

Cloning into 'lsgkm'...
remote: Enumerating objects: 96, done.
remote: Total 96 (delta 0), reused 0 (delta 0), pack-reused 96
Unpacking objects: 100% (96/96), done.
/content/lsgkm/src
g++ -Wall -Wconversion -O3 -fPIC -c libsvm.cpp
g++ -Wall -Wconversion -O3 -fPIC -c libsvm_gkm.c
g++ -Wall -Wconversion -O3 -fPIC gkmtrain.c libsvm.o libsvm_gkm.o -o gkmtrain -lm -lpthread
g++ -Wall -Wconversion -O3 -fPIC gkmpredict.c libsvm.o libsvm_gkm.o -o gkmpredict -lm -lpthread
/content


In [3]:
#get the raw data
!wget https://raw.githubusercontent.com/AvantiShri/model_storage/master/deeplift/genomics/sequences.simdata.gz


Redirecting output to ‘wget-log’.


In [4]:
!zcat sequences.simdata | head

seqName	sequence	embeddings	task1	task2	task3
empty_1732	AGCTCTTTTGGCGGACCGAGTTCGTCGGACTTTCGCTAAATTGACCATTAACAGGAGGATACCCGGGTGTCTCGGGAAGGTCGTAATACGGTAATTACGGATGTGAGAGCGCACACATCCTTTGGATGATACATGATCTCGCCGTATATGGCCTAGGTCAAAAGGACAGTCTCGCTAATCGCAATAATGCAGCTAACCGA		0	0	0
gata_only_1527	TGCAGATAAGAACTTTGGGAATATGGTAGGAAAAACCCCTGAAGTGAGATCCAAAAAATATAGACTGGCGCGGTGGCAATGCCCTTGTTAATGCTGGGAATCAAAAAATCGAAACTCTCAGCAGATAAGATATGCAGACCTGCTCGTGGAGACAAAAAGTATTGTTACTGCGTAAACCATTAGTACGTTGCCGAGTACCG	pos-120_GATA_disc1-GCAGATAAGA,pos-1_GATA_disc1-GCAGATAAGA	0	1	0
gata_tal1_62	AGATTATAGGCCCTCCACCAGATAAGATATAGCGTTCGAGGGTCAGAAGGTAGTTCTGAAGATCTCTTCATTGTGAAAGGCCTCATATGATGAGCTTCTCCATTCTCAGCTGACAGGCAAAACAATTGATCAGCGCTTAGGCGCCAGATGTTTACTCCCGTTAAGTCAACTTAGCTGTGTTCGTCAAATCCGCAACGCGG	pos-17_GATA_disc1-CCAGATAAGA,pos-139_TAL1_known1-GGCGCCAGATGTTTAC	1	1	1
empty_1289	TTTGGGCGCGGAAGAAGTCGCTCGCCTTTCTGTACTTGAAAATGATTACATGTAAGGATGTAACTCAATCGCAACATTGTCATTGGGTTCCATCGCCCTAGCGGTAGTAGGTTTTACCTCCAACTTGACAACCCTTCGTAAATATAAAATAGTAGTAA

In [0]:
!zcat sequences.simdata.gz | perl -lane 'if ($F[3] eq "1") {print ">$F[0]\n$F[1]"}' > positives.fa
!zcat sequences.simdata.gz | perl -lane 'if ($F[3] eq "0") {print ">$F[0]\n$F[1]"}' > negatives.fa
!perl -ne 'if ($.%10 == 1 || $.%10 == 2) {print $_}' positives.fa > test_positives.fa
!perl -ne 'if ($.%10 != 1 && $.%10 != 2) {print $_}' positives.fa > train_positives.fa
!perl -ne 'if ($.%10 == 1 || $.%10 == 2) {print $_}' negatives.fa > test_negatives.fa
!perl -ne 'if ($.%10 != 1 && $.%10 != 2) {print $_}' negatives.fa > train_negatives.fa

In [6]:
!wc -l *.fa

  12000 negatives.fa
   4000 positives.fa
   2400 test_negatives.fa
    800 test_positives.fa
   9600 train_negatives.fa
   3200 train_positives.fa
  32000 total


In [7]:
!lsgkm/src/gkmtrain


Usage: gkmtrain [options] <posfile> <negfile> <outprefix>

 train gkm-SVM using libSVM

Arguments:
 posfile: positive sequence file (FASTA format)
 negfile: negative sequence file (FASTA format)
 outprefix: prefix of output file(s) <outprefix>.model.txt or
            <outprefix>.cvpred.txt

Options:
 -t <0 ~ 5>   set kernel function (default: 4 wgkm)
              NOTE: RBF kernels (3 and 5) work best with -c 10 -g 2
                0 -- gapped-kmer
                1 -- estimated l-mer with full filter
                2 -- estimated l-mer with truncated filter (gkm)
                3 -- gkm + RBF (gkmrbf)
                4 -- gkm + center weighted (wgkm)
                     [weight = max(M, floor(M*exp(-ln(2)*D/H)+1))]
                5 -- gkm + center weighted + RBF (wgkmrbf)
 -l <int>     set word length, 3<=l<=12 (default: 11)
 -k <int>     set number of informative column, k<=l (default: 7)
 -d <int>     set maximum number of mismatches to consider, d<=4 (default: 3)
 -g <float>

In [9]:
import time

t=3
c=10
g=2
l=6
k=4
d=0

!lsgkm/src/gkmtrain -t $t -c $10 -g $2 -l $l -k $k -d $d -w 3 train_positives.fa train_negatives.fa "params_t"$t"_l"$l"_k"$k"_d"$d"_g"$g"_c"$c"_w3"


start = time.time()
!lsgkm/src/gkmpredict test_positives.fa "params_t"$t"_l"$l"_k"$k"_d"$d"_g"$g"_c"$c"_w3.model.txt" preds_positives.txt
end = time.time()
print("Time taken per seq:",(end-start)/400)
start = time.time()
!lsgkm/src/gkmpredict test_negatives.fa "params_t"$t"_l"$l"_k"$k"_d"$d"_g"$g"_c"$c"_w3.model.txt" preds_negatives.txt
end = time.time()
print("Time taken per seq:",(end-start)/1200)

import numpy as np
from sklearn.metrics import roc_auc_score
pos_preds = [float(x.rstrip().split("\t")[1]) for x in open("preds_positives.txt")]
neg_preds = [float(x.rstrip().split("\t")[1]) for x in open("preds_negatives.txt")]
print(roc_auc_score(y_true=np.array([1 for i in range(len(pos_preds))]+[0 for i in range(len(neg_preds))]),
                    y_score=np.array(pos_preds+neg_preds)))

INFO 2018-09-25 19:07:27: Number of threads is set to 1
INFO 2018-09-25 19:07:27: Arguments:
INFO 2018-09-25 19:07:27:   posfile = train_positives.fa
INFO 2018-09-25 19:07:27:   negfile = train_negatives.fa
INFO 2018-09-25 19:07:27:   outprefix = params_t3_l6_k4_d0_g2_c10_w3
INFO 2018-09-25 19:07:27: Parameters:
INFO 2018-09-25 19:07:27:   kernel-type = 3
INFO 2018-09-25 19:07:27:   L = 6
INFO 2018-09-25 19:07:27:   k = 4
INFO 2018-09-25 19:07:27:   d = 0
INFO 2018-09-25 19:07:27:   gamma = 2
INFO 2018-09-25 19:07:27:   C = 10
INFO 2018-09-25 19:07:27:   w = 3
INFO 2018-09-25 19:07:27:   eps = 0.001
INFO 2018-09-25 19:07:27:   shrinking = no
INFO 2018-09-25 19:07:27: reading 1600 sequences from train_positives.fa
INFO 2018-09-25 19:07:27: reading... 0
INFO 2018-09-25 19:07:27: reading... 1000
INFO 2018-09-25 19:07:27: reading... done
INFO 2018-09-25 19:07:27: reading 4800 sequences from train_negatives.fa
INFO 2018-09-25 19:07:27: reading... 0
INFO 2018-09-25 19:07:27: reading... 1000


In [12]:
!head -20 params_t3_l6_k4_d0_g2_c10_w3.model.txt

svm_type c_svc
kernel_type gkmrbf
L 6
k 4
d 0
gamma 2
nr_class 2
total_sv 4660
rho 1.57052
label 1 -1
nr_sv 1500 3160
SV
2.03351018676311 TGTTTATATCTGCCTGCGATGAGTAAATTGTCTCTTCTGAGCCAGCGGCCAGATGGTCGGGAGACTTCGGGTCATAACGATGACTTTTATATGCTGGCCAGATAACCAACCGATATTAACAAGTGAGCAACGTCACCTCGAAACGTGCCTAAAGGTCCCCCAGTCTTCCAGCTAGTCTCAGGAGATAATCCGGCGGAATG
1.632791235035324 CTTGTATTGAGTTAGAAACCGCCACGGCACTGCTATGTATGACATTCTAACTAAGTGAGTTATGCGTTGGGTCCTTTATGTGGCATTATCTGGTAATACTTAATTGATGTACTATTTCCTCGACAAAACAGGTGGTGTGGGTGTACCGGTCACCGATAAGGGGGAACTCACCAGATGGTAGTTAACCTATAGAGTCCTGA
1.357797810577622 TTCTACTTCACCCAGAATACAGCCATCAGCAATCAGCGAACAGATGTTAACTTCTAATAATAGATATGACGATGTTCAACGAGTTTATCCCCCGGATAGACACTACTCAAGCGGATCTTGTTCTTAGGTAGGATGCCGTCCACAAGTGACGTGCCATGATACAGATAAGAACCTAAGACTCCCCCAAAAGATCAGAAATT
2.285654642560899 GCGACAAACAGGTGGCGGGCGGAGTCGGCTGAGCTATTCTCGTTTAGAATGCTTAGGTTGGCCGCTCAGAGTCCTAATGCCCTGATAAGGCGACTTTCAAACACGATCAGATGGTCAGCGACTCGGCATAATACCTTAGCTCCACAGAAAGTCGGCACCTAAGCGCGTCTACTATTGCGAGCTACCTGGAGCGTAAGAGG
1.07

In [0]:
import numpy as np
from collections import Counter
import itertools


class GkmSvmModel(object):
  
  letter_to_num = {'A':0, 'C': 1, 'G': 2, 'T': 3}
  
  def __init__(self, kernel_type, l, k, d, gamma, string_svs, signed_alphas):
    self.kernel_type = kernel_type
    self.l = l
    self.k = k
    self.d = d
    self.gamma = gamma
    self.string_svs = string_svs
    self.signed_alphas = signed_alphas
    self.build()
    
  def build(self):
    self.build_nongap_pos_sets()
    self.gkmer_count_svs = [self.get_gkmer_counts(x) for x in self.string_svs]
    
  def build_nongap_pos_sets(self):
    #get the placement of the non-gap positions
    redundant_nongap_pos_sets = list(itertools.combinations(range(self.l),
                                                            self.k))
    non_redundant_nongap_pos_sets = []
    string_reps_non_redundant_nongap_pos_sets = set()
    for nongap_pos_set in redundant_nongap_pos_sets:
      string_rep_arr = [" " for i in range(self.l)]
      for a_pos in nongap_pos_set:
        string_rep_arr[a_pos] = "X"
      string_rep = "".join(string_rep_arr)
      string_rep = string_rep.lstrip().rstrip()
      if (string_rep in string_reps_non_redundant_nongap_pos_sets):
        pass
        #print("Skipping",string_rep)
      else:
        string_reps_non_redundant_nongap_pos_sets.add(string_rep)
        non_redundant_nongap_pos_sets.append(nongap_pos_set)
    self.nongap_pos_sets = non_redundant_nongap_pos_sets 
  
  def get_gkmer_counts(self, a_string):
    number_string = self.encode_as_numbers(a_string)
    k = self.k
    nongap_pos_sets = self.nongap_pos_sets
    to_count_up = []
    for i in range(len(number_string)+1-l):
      for (nongap_pos_set_num,nongap_pos_set)\
        in enumerate(nongap_pos_sets):
        gkmer = 0
        for nongap_pos_enumidx, nongap_pos in enumerate(nongap_pos_set):
          #multiply the digit by the appropriate place value given by
          #nongap_pos_enumidx, add together
          gkmer |= number_string[i+nongap_pos] << (nongap_pos_enumidx << 1)
        #(1 << (k << 1)) is the same as 4**k
        gkmer += (nongap_pos_set_num*(1 << (k << 1)))
        to_count_up.append(gkmer)
    return Counter(to_count_up)
    
  def encode_as_numbers(self, a_string):
    return [self.letter_to_num[x] for x in a_string]
    
  
  @classmethod
  def from_txt_file(cls, txt_file):
    lines = [x for x in open(txt_file)]
    
    field_name, kernel_type = lines[1].split()
    assert field_name=="kernel_type"
    field_name, l = lines[2].split()
    assert field_name=="L"; l = int(l)
    field_name, k = lines[3].split()
    assert field_name=="k"; k = int(k)
    field_name, d = lines[4].split()
    assert field_name=="d"; d = int(d)
    field_name, gamma = lines[5].split()
    assert field_name=="gamma"; gamma = float(gamma)
    
    string_svs = []
    signed_alphas = []
    assert lines[11].rstrip() == "SV"
    for a_line in lines[12:]:
      the_alpha, the_string = a_line.split()
      the_alpha = float(the_alpha)
      string_svs.append(the_string)
      signed_alphas.append(the_alpha)

    return cls(kernel_type=kernel_type,
        l=l, k=k, d=d, gamma=gamma, string_svs=string_svs,
        signed_alphas=np.array(signed_alphas))
  
  
gkm_svm_model = GkmSvmModel.from_txt_file("params_t3_l6_k4_d0_g2_c10_w3.model.txt")

#scan a sequence to get all the kmers
#map kmer --> gapped (& mismatch kmers)

In [82]:
import time
s = time.time()
gkm_svm_model.get_gkmer_counts(gkm_svm_model.string_svs[0])
print(time.time()-s)

0.006539344787597656


In [77]:
a = Counter([1,2,3,4])
a.update([1])
a

Counter({1: 2, 2: 1, 3: 1, 4: 1})

In [43]:
from functools import reduce
reduction_func = lambda x,y: x|y

def interpret_in_base_4(arr):
  return reduce(reduction_func,
                (y<<(x<<1) for (x,y) in enumerate(arr[::-1])))

bah = [3,3,2,0,3,2,1,2,1,2]

import time

start = time.time()
for i in range(10000):
  eh = bah[0] << 18| bah[1] << 16 | bah[2] << 14 | bah[3] << 12 | bah[4] << 10 | bah[5] << 8 | bah[6] << 6 | bah[7] << 4 | bah[8] << 2 | bah[9]
print(time.time()-start)

start = time.time()
for i in range(10000):
  eh = interpret_in_base_4(bah)
print(time.time()-start)


print(eh)
print(interpret_in_base_4(bah))

0.01012110710144043
0.04631161689758301
1019494
1019494


In [0]:
from collections import Counter
c = Counter()

In [0]:
c.update("a")

In [41]:
c

Counter({'a': 2})

In [47]:
oink |= 2
oink

6

In [1]:
#Let's time how long ISM would take

!wc -l test_positives.fa


wc: test_positives.fa: No such file or directory


In [59]:
!head preds_negatives.txt

empty_1732	-0.926003
tal1_only_1375	-0.719885
tal1_only_932	-0.643103
tal1_only_822	-0.311129
gata_only_1864	-0.764027
tal1_only_258	-0.170276
tal1_only_479	-0.423172
gata_only_253	-0.725754
gata_only_838	-0.581721
gata_only_1803	-0.921533


0.8409916666666667
